<a href="https://colab.research.google.com/github/adalves-ufabc/2024.Q2-PLN/blob/main/2024_Q2_PLN_AULA_13_Notebook_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

## **LangChain**
---



In [ ]:
#@title Instalando o pacote LangChain
!pip install langchain -q U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 643.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

0.2.12


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## **Dados Estruturados**

**Dados estruturados** são dados que são organizados e formatados de maneira sistemática, permitindo fácil acesso, análise e manipulação.

Frequentemente é útil ter um modelo que retorne uma saída que corresponda a um esquema específico.

**Método `.with_structured_output()`**

Esta é a maneira mais fácil e confiável de obter saídas estruturadas. O método `with_structured_output()` é implementado para modelos que fornecem APIs nativas para estruturar saídas, como chamadas de ferramentas/funções ou modo JSON, e utiliza essas capacidades internamente.

>

Este método recebe um esquema como entrada, que especifica os nomes, tipos e descrições dos atributos de saída desejados. O método retorna um `Runnable` semelhante a um modelo, exceto que, em vez de produzir strings ou Mensagens, ele produz objetos correspondentes ao esquema fornecido. O esquema pode ser especificado como uma classe `TypedDict`, um `JSON Schema` ou uma classe `Pydantic`. Se `TypedDict` ou `JSON Schema` forem usados, então um dicionário será retornado pelo `Runnable`, e se uma classe `Pydantic` for usada, então um objeto `Pydantic` será retornado.

>

Como exemplo, vamos fazer com que um modelo gere uma piada e separe a preparação do desfecho da piada:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-4o-mini")

**Classe `Pydantic`**

Se quisermos que o modelo retorne um objeto `Pydantic`, basta passar a classe `Pydantic` desejada. A principal vantagem de usar `Pydantic` é que a saída gerada pelo modelo será validada. O `Pydantic` levantará um erro se algum campo obrigatório estiver faltando ou se algum campo for do tipo errado.

In [ ]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

# Pydantic
class Piada(BaseModel):
    """Piada para contar ao usuário."""

    introducao: str = Field(description="A introdução da piada")
    arremate: str = Field(description="O desfecho da piada")
    avaliacao: Optional[int] = Field(
        default=None, description="Quão engraçada é a piada, de 1 a 10"
    )

In [ ]:
modelo_estruturado = modelo.with_structured_output(Piada)

modelo_estruturado.invoke("Me conte uma piada sobre gatos")

Piada(introducao='Por que os gatos não contam mentiras?', arremate='Porque eles sempre estão de patas dadas com a verdade!', avaliacao=7)

Além da estrutura da classe `Pydantic`, o nome da classe, a *docstring* e os nomes e as descrições fornecidas dos parâmetros são muito importantes. Na maioria das vezes, `with_structured_output` está usando a API de chamada de funções/ferramentas de um modelo, e você pode pensar efetivamente em todas essas informações como sendo adicionadas ao *prompt* do modelo.

**`TypedDict` ou `JSON Schema`**

Se você não quiser usar `Pydantic`, explicitamente não quiser validação dos argumentos ou quiser ser capaz de transmitir as saídas do modelo, pode definir seu esquema usando uma classe `TypedDict`. Opcionalmente, podemos usar uma sintaxe especial `Annotated` suportada pelo **LangChain**, que permite especificar o valor padrão e a descrição de um campo. Observe que o valor padrão não é preenchido automaticamente se o modelo não o gerar; ele é usado apenas na definição do esquema que é passado para o modelo.

In [ ]:
from typing_extensions import Annotated, TypedDict

# TypedDict
class Piada(TypedDict):
    """Piada para contar ao usuário."""

    introducao: Annotated[str, ..., "introducao"]
    arremate: Annotated[str, ..., "O desfecho da piada"]
    avaliacao: Annotated[Optional[int], None, "Quão engraçada é a piada, de 1 a 10"]

In [ ]:
modelo_estruturado = modelo.with_structured_output(Piada)

modelo_estruturado.invoke("Me conte uma piada sobre gatos")

{'introducao': 'Por que os gatos não jogam poker na selva?',
 'arremate': 'Porque há muitos cheetahs!',
 'avaliacao': 8}

De maneira equivalente, podemos passar um dicionário `JSON Schema`. Isso não requer importações ou classes e torna muito claro como cada parâmetro é documentado, com o custo de ser um pouco mais verboso.

In [ ]:
json_schema = {
    "title": "piada",
    "description": "Piada para contar ao usuário.",
    "type": "object",
    "properties": {
        "introducao": {
            "type": "string",
            "description": "A introdução da piada",
        },
        "arremate": {
            "type": "string",
            "description": "O desfecho da piada",
        },
        "avaliacao": {
            "type": "integer",
            "description": "Quão engraçada é a piada, de 1 a 10",
            "default": None,
        },
    },
    "required": ["introducao", "arremate"],
}


In [ ]:
modelo_estruturado = modelo.with_structured_output(json_schema)

modelo_estruturado.invoke("Me conte uma piada sobre gatos")

{'introducao': 'Por que os gatos não jogam cartas?',
 'arremate': 'Porque eles têm medo dos baralhos!',
 'avaliacao': 7}

**Escolhendo entre múltiplos esquemas**

A maneira mais simples de permitir que o modelo escolha entre vários esquemas é criar um esquema "pai" que tenha um atributo do tipo `Union`:

In [ ]:
from typing import Union

# Pydantic

class Piada(BaseModel):
    """Piada para contar ao usuário."""

    introducao: str = Field(description="A introdução da piada")
    arremate: str = Field(description="O desfecho da piada")
    avaliacao: Optional[int] = Field(
        default=None, description="Quão engraçada é a piada, de 1 a 10"
    )

class RespostaConversacional(BaseModel):
    """Responda de maneira conversacional. Seja gentil e prestativo."""

    resposta: str = Field(description="Uma resposta conversacional à pergunta do usuário")

class Resposta(BaseModel):
    saida: Union[Piada, RespostaConversacional]

In [ ]:
modelo_estruturado = modelo.with_structured_output(Resposta)

resposta = modelo_estruturado.invoke("Me conte uma piada sobre gatos")

In [ ]:
resposta

Resposta(saida=Piada(introducao='Por que os gatos são ótimos músicos?', arremate='Porque eles têm um ótimo senso de miado!', avaliacao=7))

In [ ]:
resposta.saida

Piada(introducao='Por que os gatos são ótimos músicos?', arremate='Porque eles têm um ótimo senso de miado!', avaliacao=7)

In [ ]:
resposta.saida.introducao

'Por que os gatos são ótimos músicos?'

In [ ]:
modelo_estruturado.invoke("Como você está hoje?")

Resposta(saida=RespostaConversacional(resposta='Estou aqui pronto para ajudar! E você, como está hoje?'))

***Few-shot prompting***

***Few-shot prompting*** é uma técnica onde se fornecem alguns exemplos específicos no *prompt* para guiar o modelo na geração de respostas. Em vez de treinar o modelo com uma nova base de dados completa, você apenas inclui alguns exemplos relevantes diretamente no *prompt*.

Para esquemas mais complexos, é muito útil adicionar exemplos ao *prompt*.

A maneira mais simples e universal é adicionar exemplos a uma mensagem do sistema no *prompt*:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

sistema = """Você é um comediante hilário. Sua especialidade são piadas do tipo "knock-knock".
Retorne uma piada que tenha a introdução (a resposta para "Quem está aí?") e o arremate final (a resposta para "<introdução> quem?").

Aqui estão alguns exemplos de piadas:

exemplo_usuario: Me conte uma piada sobre gatos
exemplo_assistente: {{"introducao": "Por que os gatos não jogam cartas?", "arremate": "Porque eles têm medo dos cães que podem trapacear!", "avaliacao": 8}}

exemplo_usuario: Conte outra piada sobre cães
exemplo_assistente: {{"introducao": "Por que os cães levam uma bola para a escola?", "arremate": "Porque eles querem aprender a jogar!", "ratavaliacaoing": 7}}

exemplo_usuario: Agora sobre peixes
exemplo_assistente: {{"introducao": "Por que o peixe foi ao banco?", "arremate": "Para abrir uma conta corrente!", "avaliacao": 9}}"""

# template de prompt
prompt = ChatPromptTemplate.from_messages([("system", sistema), ("human", "{input}")])

# uso do prompt com um modelo estruturado
modelo_estruturado_few_shot = prompt | modelo_estruturado

modelo_estruturado_few_shot.invoke("Me conte uma piada sobre elefantes")

Resposta(saida=Piada(introducao='Knock, knock!', arremate='Quem está aí? Elefante. Elefante quem? Elefante que não para de pedir comida!', avaliacao=8))

Outro exemplo:

In [ ]:
# Pydantic

class Produto(BaseModel):
    """Descrição de um produto."""

    nome: str = Field(description="O nome do produto")
    descricao: str = Field(description="A descrição do produto")

In [ ]:
modelo_estruturado = modelo.with_structured_output(Produto)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

sistema = """Você é um redator de descrições de produtos experiente. Sua tarefa é criar descrições detalhadas e envolventes para produtos.
Cada descrição deve incluir informações sobre as principais características e benefícios do produto.

Aqui estão alguns exemplos de descrições:

exemplo_usuario: Descreva um smartphone moderno
exemplo_assistente: {{"nome": "Smartphone UltraX", "descricao": "O Smartphone UltraX é equipado com uma tela AMOLED de 6,7 polegadas, processador octa-core de última geração e câmera tripla de 64MP. Oferece um desempenho rápido e uma experiência de usuário excepcional com bateria de longa duração e carregamento rápido."}}

exemplo_usuario: Descreva um aspirador de pó robô
exemplo_assistente: {{"nome": "Aspirador Robô CleanPro", "descricao": "O Aspirador Robô CleanPro é ideal para manter sua casa limpa sem esforço. Com tecnologia de navegação inteligente, ele limpa todos os cantos e vem com um sistema de filtragem HEPA que captura alérgenos e poeira. Compacto e silencioso, é fácil de usar e programar."}}

exemplo_usuario: Agora sobre uma cafeteira
exemplo_assistente: {{"nome": "Cafeteira Espresso Elite", "descricao": "A Cafeteira Espresso Elite oferece uma experiência de café de qualidade barista no conforto de sua casa. Com um sistema de pressão de 15 bar, ela extrai o máximo sabor dos grãos. Possui um moinho integrado e controle de temperatura preciso para preparar o café perfeito a cada vez."}}"""

# template de prompt
prompt = ChatPromptTemplate.from_messages([("system", sistema), ("human", "{input}")])

# uso do prompt com um modelo estruturado
modelo_estruturado_few_shot = prompt | modelo_estruturado

modelo_estruturado_few_shot.invoke("Descreva um tablet moderno")

Produto(nome='Tablet ProMax', descricao='O Tablet ProMax combina um design elegante com tecnologia de ponta, apresentando uma tela de 12,9 polegadas Liquid Retina com resolução impressionante, ideal para assistir a vídeos e trabalhar em projetos criativos. Equipado com um processador potente e 8GB de RAM, ele garante desempenho fluido para multitarefas e jogos. Sua bateria de longa duração permite horas de uso contínuo, enquanto o suporte para caneta stylus e teclado magnético transforma o tablet em uma ferramenta versátil para produtividade e entretenimento.')

Para modelos que suportam mais de um meio de estruturar saídas, você pode especificar qual método usar com o argumento `method=`.

In [ ]:
modelo_estruturado = modelo.with_structured_output(None, method="json_mode")

modelo_estruturado.invoke(
    "Descreva um tablet moderno, responda em JSON com as chaves `nome` e `descricao`"
)

{'nome': 'Tablet Pro 2023',
 'descricao': 'O Tablet Pro 2023 é um dispositivo ultrafino e leve, equipado com uma tela de alta definição de 12,9 polegadas, que oferece cores vibrantes e detalhes impressionantes. Possui um processador de última geração, garantindo desempenho rápido e eficiente para multitarefas e jogos. Com 256 GB de armazenamento interno e suporte a cartão microSD, oferece amplo espaço para aplicativos, fotos e vídeos. A bateria de longa duração permite até 12 horas de uso contínuo e a conectividade 5G garante acesso rápido à internet. O tablet também suporta caneta stylus e teclado, tornando-o ideal para produtividade e criatividade. Além disso, o sistema operacional mais recente oferece uma interface intuitiva e uma vasta gama de aplicativos disponíveis na loja.'}

Modelos de linguagem não são perfeitos ao gerar saídas estruturadas, especialmente quando os esquemas se tornam complexos. Você pode evitar a geração de exceções e lidar com a saída bruta você mesmo passando `include_raw=True`. Isso altera o formato da saída para conter a mensagem bruta, o valor analisado (se bem-sucedido) e quaisquer erros resultantes.

In [ ]:
modelo_estruturado = modelo.with_structured_output(Produto, include_raw=True)

modelo_estruturado.invoke("Descreva um tablet moderno")

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DbR0a6bglkQP9XFK4QIRr22L', 'function': {'arguments': '{"nome":"Tablet Moderno","descricao":"Um tablet moderno é um dispositivo portátil e elegante, que combina a funcionalidade de um computador com a conveniência de um smartphone. Geralmente apresenta uma tela sensível ao toque de alta definição, permitindo uma experiência visual imersiva para assistir a vídeos, jogar e navegar na internet. Equipado com um processador potente e memória RAM generosa, proporciona desempenho ágil em multitarefas e execução de aplicativos exigentes. Os tablets modernos também contam com conectividade Wi-Fi e, em muitos casos, opções de conectividade 4G ou 5G, permitindo acesso à internet em qualquer lugar. Além disso, possuem câmeras de alta qualidade para fotos e videochamadas, e vêm com sistemas operacionais intuitivos, como Android ou iOS, oferecendo uma ampla gama de aplicativos para produtividade, entretenimento e criatividad

**Referência**:

> https://python.langchain.com/v0.2/docs/how_to/structured_output/